In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
from IPython.display import HTML
import math
import os
import csv
from moviepy.editor import VideoFileClip
from scipy.interpolate import UnivariateSpline

In [ ]:
def grayscale(img):
    """
    Applies the Grayscale transform
    This will return an image with only one color channel
    you should call plt.imshow(gray, cmap='gray').
    
    Input: 
    img: color image (3D array) (3 channels).
    Output: 
    gray-scale image (1 channel).
    """
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def gaussian_blur(img, kernel_size):
    """
    Applies a Gaussian filter to smooth the gray scale image.
    
    Input: 
    ima: gray scale image
    kernel_size: integer.
    Output:
    blurred gray scale image using the Gaussian Filter with specified kernel size.
    """
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def max_blur(img, kernel_size):
    """
    Morphological filtering: within the kernel, assign the center pixel the value equal to the maximum
    pixel values in the neighboring region defined by the kernel size.
    
    Input: 
    (1-channel)binary image. 
    kernel size: Integer. 
    Output: 
    gray scale binary image after the max filter.
    """
    padding = (int)(kernel_size / 2)
    replicate = cv2.copyMakeBorder(img,padding,padding,padding,padding,cv2.BORDER_CONSTANT, value = 0)
    for i in range(0, img.shape[0]):
        for j in range(0, img.shape[1]):
            img[i][j] = np.amax(replicate[i : i + kernel_size, j : j + kernel_size])
    return img
    

def region_of_interest(img, vertices, image_name):
    """
    Applies an image mask.
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    
    Input: 
    image: 2D arrays
    vertices: an interger array that stores the vertices of the polygon.
    image_name: a String that stores the name of the image.
    Output: 
    Image the same size as the input image but with pixels outside the polygon.
    defined by the vertices having value of 0.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    plt.imshow(masked_image, cmap = 'gray')
    plt.axis('off')
    plt.savefig("/Users/yuzhesheng/Desktop/CVProject/lane_detection/" + image_name + "_ROI.png")
    plt.show()
    return masked_image


def startPoints(roi_image, image_name):
    """
    Computes the starting x-coordinates (left and right) as the initial 
    x-coordinates of the centers of the sliding windows (left and right)
    based on the histogram.
    
    Input: 
    roi_image: the ROI (2D array)
    image_name: the String that stores the name of the image
    Output: 
    left_x: left initial x-coordinate of the sliding window center.
    right_x: right initial x-coordinate of the sliding window center.
    """
    histogram = np.sum(roi_image[750:900, :], axis = 0 )
    
    # find starting position
    mid = np.int(histogram.shape[0] / 2)
    left_mid = hist_mean(histogram, 0, mid)
    right_mid = hist_mean(histogram, mid, histogram.shape[0])

    left_x = np.argmax(histogram[ : left_mid])
    right_x = np.argmax(histogram[right_mid : ]) + right_mid

    plt.imshow(roi_image,cmap = 'gray')
    plt.axvline(x=left_x,color='r', linestyle='--', lw = 5)
    plt.axvline(x=right_x,color='r', linestyle='--', lw = 5)
    plt.savefig("/Users/yuzhesheng/Desktop/CVProject/lane_detection/" + image_name + "ROI_Start.png")
    plt.show()
    
    plt.plot(histogram)
    plt.plot(left_x, histogram[left_x], 'ro', ms = 10)
    plt.plot(right_x, histogram[right_x], 'ro', ms =10)
    plt.savefig("/Users/yuzhesheng/Desktop/CVProject/lane_detection/" + image_name + "_Histogram.png")
    plt.show()
    
    return left_x, right_x

def hist_mean(histogram, start, end):
    """
    calculate the average location (x-coordinate) between start and end
    based on the histogram.
    
    Input: 
    histogram: each x value has a bin, representing the number of times 
    the x value appears. 
    start: Integer that denotes the lower boundary of search area.
    end: Integer that denotes the upper boundary of search area.
    Output: 
    the average x location (integer) between start and end.
    """
    count = 0
    summation = 0
    for x in range(start, end):
        count = count + histogram[x]
        summation = summation + x * histogram[x]
    return np.int(summation / count)

In [ ]:
# sliding window with changing window size
def slidingWindow(image, roi_image, left_x, right_x, n_windows, a, image_name):
    """
    detect the all the pixels belonging to the lane markings (left and right)
    For each window, find the direction in which the following window should be
    moved. Then model all nonzero pixels inside that window as the lane marking pixels.
    
    Input: 
    image: original image that has 3 channels.
    roi_image: retaining the pixels values within polygon
    left_x: (integer) the initial x coordinate of the left sliding window center 
    right_x: (integer) the initial x coordinate of the right sliding window center 
    n_window: (integer) the number of windows
    a: (float) the momentum term that is used to control how much further the following window 
    will be moved according to the location difference (When the x-coordinate of the next
    window center is delta_x away from the current x-coordinate of the sliding window, move the 
    next window FURTHER by a*delta_x).
    image_name: the String that stores the name of the image.
    Output: leftx: (integer array) the x-coordinates of all the pixels belonging to the
    left lane markings.
    lefty: (integer array) the y-coordinates of all the pixels belonging to the
    left lane markings
    rightx: (integer array) the x-coordinates of all the pixels belonging to the
    right lane markings.
    righty: (integer array) the y-coordinates of all the pixels belonging to the
    right lane markings
    """
    imshape = roi_image.shape
    nwindow = n_windows
    
    window_start = imshape[0] - int(imshape[0]/5)
    
    window_height = int((window_start - (imshape[0]* 2 / 7 + 80)) / nwindow)

    margin_left = 100
    margin_right = 100

    minpix_top = 9
    minpix_window = 10

    nonzero = roi_image.nonzero()
    nonzeroy = np.array(nonzero[0]) # nonzeroy stores the y-coordinates of the nonzero pixels
    nonzerox = np.array(nonzero[1]) # nonzerox stores the x-coordinates of the nonzero pixels

    curr_left_x = left_x
    curr_right_x = right_x
    left_lane_inds = []
    right_lane_inds = []
    
    out_image = np.dstack((roi_image, roi_image, roi_image)) * 255
  
    deltaX_left = 0
    deltaX_right = 0
    
    for window in range(nwindow):
        
        # Identify window boundaries in x and y (and right and left)
        win_y_low = window_start - (window+1)*window_height
        win_y_high = window_start - window*window_height
        win_xleft_low = curr_left_x - margin_left 
        win_xleft_high = curr_left_x + margin_left 
        win_xright_low = curr_right_x - margin_right 
        win_xright_high = curr_right_x + margin_right
        
        # Draw the windows on the visualization image    
        cv2.rectangle(out_image,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),
        (0,255,0), 2)
        cv2.rectangle(out_image,(win_xright_low,win_y_low),(win_xright_high,win_y_high)
        ,(0,255,0), 2)

        # Identify the nonzero pixels in x and y within the window and on the top of the window
        good_left_inds_top = ((nonzeroy == win_y_low) &
        (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        #print(good_left_inds.shape)
        good_right_inds_top = ((nonzeroy == win_y_low) &
        (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]

        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) &
        (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        #print(good_left_inds.shape)
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) &
        (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        #print(good_right_inds)

        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        deltaX_left = curr_left_x
        deltaX_right = curr_right_x
        
        # If you found the pixel on top of the window > minpix pixels, 
        # recenter next window on the mean position of the top window pixels
        if len(good_left_inds) < minpix_window and len(good_left_inds_top) > minpix_top:
            curr_left_x = np.int(np.mean(nonzerox[good_left_inds_top]))
        # if the not enough pixels found on top, use the mean of all pixels inside the window
        elif len(good_left_inds) > minpix_window and len(good_left_inds_top) < minpix_top:
            curr_left_x = np.int(np.mean(nonzerox[good_left_inds]))
            margin_left = margin_left - 8
        elif len(good_left_inds) > minpix_window and len(good_left_inds_top) > minpix_top:
            curr_left_x = np.int(np.mean(nonzerox[good_left_inds_top]))

        if len(good_right_inds) < minpix_window and len(good_right_inds_top) > minpix_top:
            curr_right_x = np.int(np.mean(nonzerox[good_right_inds_top]))
        elif len(good_right_inds) > minpix_window and len(good_right_inds_top) < minpix_top:
            curr_right_x = np.int(np.mean(nonzerox[good_right_inds]))
            margin_right = margin_right - 8
        elif len(good_right_inds) > minpix_window and len(good_right_inds_top) > minpix_top:
            curr_right_x = np.int(np.mean(nonzerox[good_right_inds_top]))
        
        # add a momentum term to control how much FURTHER the window should be moved.
        curr_left_x = (int)(curr_left_x + a * (curr_left_x - deltaX_left))
        curr_right_x = (int)(curr_right_x + a * (curr_right_x - deltaX_right))
        
        # shrink the window width
        if len(good_left_inds) > 0:
            margin_left = margin_left - int(80 / n_windows)
        if len(good_right_inds) > 0:
            margin_right = margin_right - int(80 / n_windows)
        
        # reduce the threshold on the number of pixels
        minpix_window = minpix_window - int(5 / n_windows)
        minpix_top = minpix_top - int(5 / n_windows)
    
    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)
    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds].astype(int)
    lefty = nonzeroy[left_lane_inds].astype(int)
    rightx = nonzerox[right_lane_inds].astype(int)
    righty = nonzeroy[right_lane_inds].astype(int)
    out_image[lefty, leftx] = [255, 0, 0]
    out_image[righty, rightx] = [0, 0, 255]
    plt.imshow(out_image)
    filename = image_name + "_window"
    plt.axis('off')
    plt.savefig("/Users/yuzhesheng/Desktop/CVProject/lane_detection/" + filename + ".png")
    plt.show()
    return leftx, lefty, rightx, righty

In [ ]:
def lane_Decision(image, leftx, lefty, rightx, righty, image_name):
    """
    For the detected coordinates of x and y from sliding windows, for each y, 
    take the MEDIAN of x position as the x-coordinate at that corresponding y-coordinate.
    
    Input: 
    image: original 3 channel color image to be drawn on. 
    leftx: (integer array) the x-coordinates of all the pixels belonging to the
    left lane markings.
    lefty: (integer array) the y-coordinates of all the pixels belonging to the
    left lane markings.
    rightx: (integer array) the x-coordinates of all the pixels belonging to the
    right lane markings.
    righty: (integer array) the y-coordinates of all the pixels belonging to the
    right lane markings.
    image_name: A String that stores the name of the image. 
    Output: 
    pos_xleft: the median, or the spline interpolated x-coordinates of detected 
    lane markings associated with each left y-coordinate.
    Y_left: an increasing interger array that stores all y-coordinates bounded by lefty.
    pos_xright: the median, or the spline interpolated x-coordinates of detected 
    lane markings associated with each right y-coordinate.
    Y_right: an increasing interger array that stores all y-coordinates bounded by righty.
    """
    Y_left = np.arange(lefty[-1], lefty[0] + 1, 1, dtype = np.int)
    Y_right = np.arange(righty[-1], righty[0] + 1, 1, dtype = np.int)
    
    # find the median of the x-coordinates corresponding to each detected y-coordinate
    pos_xleft, pos_yleft = median_PosX(leftx, lefty, Y_left, 0) # float
    pos_xright, pos_yright = median_PosX(rightx, righty, Y_right, 0) # float
    
    # Univaraite Spline fit for each detected pixels for the left and right
    # lane markings
    if len(pos_yleft) >= 4:
        spline_left = UnivariateSpline(pos_yleft, pos_xleft)
        pos_xleft = spline_left(Y_left)
    
    if len(pos_yright) >= 4:
        spline_right = UnivariateSpline(pos_yright, pos_xright)
        pos_xright = spline_right(Y_right)
    
    plt.imshow(image)
    if len(pos_yleft) >= 4:
        plt.plot(pos_xleft, Y_left, 'ro', c='r', ms = 5)
    else:
        plt.plot(pos_xleft, pos_yleft, 'ro', c='b',ms = 5)
    
    if len(pos_yright) >= 4:
        plt.plot(pos_xright, Y_right, 'ro', c= 'r', ms = 5)
    else:
        plt.plot(pos_xright, pos_yright, 'ro', c='b',ms = 5)
        
    filename = image_name + "_detectedlanepixels"
    plt.axis('off')
    plt.savefig("/Users/yuzhesheng/Desktop/CVProject/lane_detection/" + filename + ".png")
    plt.show()
    
    return pos_xleft, Y_left, pos_xright, Y_right

In [ ]:
def evaluation(image, pos_xleft, Y_left, pos_xright, Y_right, image_name):
    """
    Evaluate the performance of the algorthm. First find the Reeve's Index (FPC) of each of the 
    left and right lane markings. If FPC > 0.7, then the detection is considered as successful.
    For successful detection, find the maximum x-coordinate between the the detected pixels and the 
    ground truth. 
    
    Input:
    image: original image to be drawn on.
    pos_xleft: Float array that stores the x coordinates of the detected left lane for each y.
    Y_left: an increasing interger array that stores all y-coordinates bounded by detected left
    y locations.
    pos_xright: Float array that stores the x coordinates of the detected right lane for each y.
    Y_right: an increasing interger array that stores all y-coordinates bounded by detected right
    y locations.
    image_name: a String that stores the name of the image. 
    Output:
    An array of String that stores the name of the image, the FPC for the left and right lane 
    markings, the FP rate for the left and right lane markings, the FN rate for the left
    and right lane markings, the x-coordinate maximum displacement, and whether the detection is 
    successful or not depending on the FPC
    """
    true_left = []
    true_right = []
    status = "Success"
    scores = [] #[FPC_L, FPC_R, FP_L, FP_R, FN_L, FN_R, precision, success?]
    # read in the x and y coordinates of the labeled ground truth. 
    file_l = image_name + "_l.csv"
    file_r = image_name + "_r.csv"
    filename = image_name + "_groundtruthlabel"
    with open(file_l) as csvDataFile:
        csvReader = csv.reader(csvDataFile)
        for row in csvReader:
            true_left.append(row)
            
    with open(file_r) as csvDataFile:
        csvReader = csv.reader(csvDataFile)
        for row in csvReader:
            true_right.append(row)
  
    true_left = np.array(true_left).astype(float).astype(int)
    true_right = np.array(true_right).astype(float).astype(int)
    plt.imshow(image)
    plt.plot(true_left[:,0], true_left[:,1], 'ro', c = 'g', ms = 5)
    plt.plot(true_right[:,0], true_right[:,1], 'ro', c = 'r', ms=5)
    plt.axis('off')
    plt.savefig("/Users/yuzhesheng/Desktop/CVProject/lane_detection/" + filename + ".png")
    plt.show()
    
    # the range of the y-coordinate from the ground truth
    true_ylmin = np.amin(true_left[:,1] )
    true_ylmax = np.amax(true_left[:,1])
    true_yrmin = np.amin(true_right[:,1])
    true_yrmax = np.amax(true_right[:,1])

    # sorted array that stores the y coordinate for the ground truth
    Y_trueL = np.arange(true_ylmin, true_ylmax + 1, 1)
    Y_trueR = np.arange(true_yrmin, true_yrmax + 1, 1)
    
    # For each sorted array of y, find the corresponding x coordinate from ground truth
    X_trueL, Y_trueL = median_PosX(true_left[:,0], true_left[:,1], Y_trueL, 1) # float
    X_trueR, Y_trueR = median_PosX(true_right[:,0], true_right[:,1], Y_trueR, 1) # float
    
    #fill the gaps using linear interpolation 
    fill_MissingPoint(X_trueL, Y_trueL)
    fill_MissingPoint(X_trueR, Y_trueR)
    
    plt.imshow(image)
    percentage = 0.95
    if (true_left.shape[0] >= 5):
        start_ind =int(X_trueL.shape[0]*(1-percentage))
        X_trueL = X_trueL[start_ind:]
        Y_trueL = Y_trueL[start_ind:]
        plt.plot(X_trueL, Y_trueL, '*', c='w', ms = 5)
    
    if (true_right.shape[0] >= 5):
        start_ind =int(X_trueR.shape[0]*(1-percentage))
        X_trueR = X_trueR[start_ind:]
        Y_trueR = Y_trueR[start_ind:]
        plt.plot(X_trueR, Y_trueR, '*', c='w', ms = 5)
    
    plt.show()
    # detect whether the length of the detected lanes have exceeded 70% of the
    # label lanes lengths.
    FP_L, FN_L, FPC_L = similarity_measure(Y_left, Y_trueL)
    FP_R, FN_R, FPC_R = similarity_measure(Y_right, Y_trueR)
    
    print("False_positive rate for the left lane is: ", FP_L)
    print("False_negative rate for the left lane is: ", FN_L)
    print("FPC for the left lane is: ", FPC_L)
    print("False_positive rate for the right lane is: ", FP_R)
    print("False_negative rate for the right lane is: ", FN_R)
    print("FPC for the right lane is: ", FPC_R)
    
    # check whether the detection is successful
    if FPC_L < 0.7 and FPC_R < 0.7:
        print("FPC for both lanes too low: FAILED")
        status = "Both failed"
    elif FPC_L < 0.7:
        print("FPC for the left lane too low: FAILED")
        status = "Left failed"
    elif FPC_R < 0.7:
        print("FPC for the right lane too low: FAILED")
        status = "Right failed"
        
    left_xdistance = 0
    right_xdistance = 0
    
    # find the maximum displacement of x coordinate for the left and
    # right lane markings. 
    for i in range(X_trueL.shape[0]):
        if Y_trueL[i] >=Y_left[0] and Y_trueL[i] <= Y_left[-1]:
            ind = np.where(Y_left == Y_trueL[i])
            left_xdistance = max(left_xdistance, abs(X_trueL[i] - pos_xleft[ind])[0])
    
    for i in range(X_trueR.shape[0]):
        if Y_trueR[i] >=Y_right[0] and Y_trueR[i] <= Y_right[-1]:
            ind = np.where(Y_right == Y_trueR[i])
            right_xdistance = max(right_xdistance, abs(X_trueR[i] - pos_xright[ind])[0])
            
    precision = max(left_xdistance, right_xdistance)
    scores.append([image_name[3:], str(FPC_L), str(FPC_R), str(FP_L), str(FP_R), str(FN_L), str(FN_R), str(precision), status])
    ret = np.array(scores).reshape(1,9)
    return ret

In [ ]:
def similarity_measure(Y1, Y2):
    """
    Y1 denotes the detected height range, and Y2 denotes the Ground truth height range
    similarity_measure calculates the fraction of pixels correct (FPC or the Reeves Index)
    FPC = (|Y2| - |Y2 intersects (not Y1)| - |(not Y2) intersects Y1|)/(Y2)
    
    Input: 
    Y1: detected height range stored in an Integer array (sorted)
    Y2: ground truth height range stored in an Integer array (sorted)
    Output:
    FP: the ratio between the number of FP and the ground truth height
    FN: the ratio between the number of FN and the ground truth height
    FPC: the Reeves index
    """
    
    # FP pixel number
    False_pos = max(0, Y2[0] - Y1[0]) + max(0, Y1[-1] - Y2[-1])
    # FN pixel number
    False_neg = max(0, Y1[0] - Y2[0]) + max(0, Y2[-1] - Y1[-1])
    True_height = Y2[-1] - Y2[0] + 1
    
    if Y1[-1] < Y2[0] or Y1[0] > Y2[-1]:
        False_pos = Y1[-1] - Y1[0] + 1
        False_neg = True_height

    FP = np.float(False_pos / True_height)
    FN = np.float(False_neg / True_height)
    FPC = np.float((True_height - False_pos - False_neg) / True_height)
    return FP, FN, FPC

In [ ]:
def median_PosX(X, Y, Range, flag):
    """
    For each element r in Range, find the MEDIAN x position corresponding to that r (y-coordinate)
    there is any. Store -1 if there is no that specific y coordinate for the ground truth ONLY. For 
    the detected pixels, do NOT store -1 in that missing y coordinate
    
    Input: 
    X: Integer array that stores ALL the detected x coordinates
    Y: Integer array that stores ALL the detected y coordinates
    Range: Sorted Integer array that stores all interger y coordinate bounded by Y
    flag: 0 or 1. 0: do not insert -1 for missing y position. 1: insert -1 for missing y
    Output:
    pos_x: Integer array that stores the MEDIAN x-coordinate 
    or -1 for each y-coordinate (-1 if and only if no x-coordinates
    are detected for that y value).
    pos_y: Integer array that stores all y coordinates
    """
    status = flag
    pos_x = []
    pos_y = []
    for y in Range:
        inds = (Y == y).nonzero()[0]
        if len(inds) > 0:
            pos_x.append(np.float(np.median(X[inds])))
            pos_y.append(y)
        elif flag == 1:
            pos_x.append(-1.0)
            pos_y.append(y)
            
    pos_x = np.array(pos_x)
    pos_y = np.array(pos_y)
    
    return pos_x, pos_y

In [ ]:
def fill_MissingPoint(X, Y):
    """
    Fill in the missing x coordinates if there is any by linear interpolation.
    
    Input:
    X: Integer array that stores the x-coordinates or -1 (when no x locations are 
    detected from that y)
    Y: All y coordinates 
    """
    i = 0
    start = 0
    end = 0
    while (i < X.shape[0]):
        # No x coordinates are found for this y coordinate
        if X[i] == -1: 
            start = i - 1 # start of the linear interpolation point
            while (i < X.shape[0] and X[i] == -1):
                i = i + 1
            end = i # end of the linear interpolation point 
            for j in range(start+1, end):
                X[j] = float(X[start] + (X[end] - X[start]) / (Y[end] - Y[start]) * (Y[j] - Y[start]))
        else:
            i = i + 1

In [ ]:
def process_image(image, image_name):
    """
    Process the image. First apply yellow and white mask. Then threshold the image to obtain
    an enhanced lane marking image. Then apply the maximum filter to connect broken pieces of lane
    markings. Then find the ROI of the filtered image given a specified vertice array. Then use sliding
    window method to detect all pixels associated with the left lane and right lane pixels.
    
    Input:
    image: colored image
    image_name: a String that stores the name of the image
    Output:
    same as lane_decision function 
    pos_xleft, Y_left, pos_xright, Y_right that stores the x y coordinates of the left and right lane
    markings. 
    """
    global first_frame
    
    first_frame = 1
    plt.imshow(image)
    plt.show()
    
    imshape = image.shape
    # vertices of the ROI
    lower_left = [imshape[1]/20,imshape[0]-imshape[0]/6]
    lower_right = [imshape[1] - imshape[1]/20,imshape[0]-imshape[0]/6]
    top_left = [imshape[1]/2-imshape[1]/10,imshape[0]/4+imshape[0]/12]
    top_right = [imshape[1]/2+imshape[1]/10,imshape[0]/4+imshape[0]/12]
    vertices = [np.array([lower_left,top_left,top_right,lower_right],dtype=np.int32)]

    roi_image = region_of_interest(image, vertices, image_name)
    gray_image = grayscale(roi_image)
    
    # convert the colored image to HSV 
    img_hsv = cv2.cvtColor(roi_image, cv2.COLOR_RGB2HSV)
    h,s,v = cv2.split(img_hsv)
    
    # threshold for the yellow color
    lower_yellow = np.array([0, 100, 170], dtype = "uint8")
    upper_yellow = np.array([255, 240, 255], dtype="uint8")
    
    # apply the yellow and white mask to detect lanes
    mask_yellow = cv2.inRange(img_hsv, lower_yellow, upper_yellow)
    mask_white = cv2.inRange(gray_image, 210, 255)
    mask_yw = cv2.bitwise_or(mask_white, mask_yellow)

    kernel_size = 9
    max_gray = max_blur(mask_yw,kernel_size)
    max_gray = cv2.medianBlur(max_gray, kernel_size)
    plt.imshow(max_gray, cmap='gray')
    plt.axis('off')
    filename = image_name + "_max_filter"
    plt.savefig("/Users/yuzhesheng/Desktop/CVProject/lane_detection/" + filename + ".png")
    plt.show()
    
    left_x, right_x = startPoints(max_gray, image_name)
    leftx, lefty, rightx, righty = slidingWindow(image, max_gray, left_x, right_x, 30, 0.95, image_name)
    if len(lefty) < 20 or len(righty) < 20:
        print(image_name + "failed because no lanes are detected")
        return 
    
    return lane_Decision(image, leftx, lefty, rightx, righty, image_name)

In [ ]:
Total_score = []
Total_score.append(["Image_id", "FPC_L","FPC_R","FP_L","FP_R","FN_L","FN_R", "Precision", "succeed?"])
Total_score = np.array(Total_score).reshape(1,9)
for i in np.arange(10, 71, 10):
    foldername = str(i) + "/"
    for j in range (1, 11):
        image_name = foldername + str(i - 10 + j)
        image = mpimg.imread(image_name + ".jpg")
        pos_xleft, Y_left, pos_xright, Y_right = process_image(image, image_name)
        
        if pos_xleft is None or pos_xright is None or pos_xright.shape[0] < 20 or pos_xleft.shape[0] < 20 or Y_left is None or Y_right is None:
            continue
            
        score = evaluation(image, pos_xleft, Y_left, pos_xright, Y_right, image_name)
        Total_score = np.concatenate((Total_score, score), axis=0)

In [ ]:
print(Total_score)
with open("0_70.csv",'w') as file:
    wr = csv.writer(file)
    for row in Total_score:
        wr.writerow(row)